# Language Modeling: Character level
___
#### Description: 
In this notebook I use a recurrent neural network (RNN) to generate new Pokemon names. The model is trained using a many-to-one RNN where the input is a sequence of characters and the output is the character that follows the sequence. 
___
#### Dataset: 
The dataset contains all 807 Pokemon names.
___
#### Reference: 

This notebook was completed using the following resources as a guide:

https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/

https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

In [2]:
# Get data
import pandas as pd
df = pd.read_csv('Datasets/pokemon.csv')
df.head(3)

,index,name
0,1,Bulbasaur
1,2,Ivysaur
2,3,Venusaur


In [4]:
# Extract names from dataframe
import numpy as np

# List containing all names in lowercase with a newline appended at end
names = [name.lower()+'\n' for name in df['name']]

In [7]:
# The newline indicates when a name ends
names[0:10]

['bulbasaur\n',
 'ivysaur\n',
 'venusaur\n',
 'charmander\n',
 'charmeleon\n',
 'charizard\n',
 'squirtle\n',
 'wartortle\n',
 'blastoise\n',
 'caterpie\n']

In [8]:
# Extract all unique characters from names
all_names = ''.join(names)
chars = sorted(set(all_names))
print(chars)

['\n', ' ', '-', '.', '2', ':', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '’', '♀', '♂']


In [10]:
# Create dictionaries that maps characters to indices and vice versa
char_indices = dict((c, i+1) for i, c in enumerate(chars))
indices_char = dict((i+1, c) for i, c in enumerate(chars))

In [14]:
# 0th index reserved for padding
indices_char

{1: '\n',
 2: ' ',
 3: '-',
 4: '.',
 5: '2',
 6: ':',
 7: 'a',
 8: 'b',
 9: 'c',
 10: 'd',
 11: 'e',
 12: 'f',
 13: 'g',
 14: 'h',
 15: 'i',
 16: 'j',
 17: 'k',
 18: 'l',
 19: 'm',
 20: 'n',
 21: 'o',
 22: 'p',
 23: 'q',
 24: 'r',
 25: 's',
 26: 't',
 27: 'u',
 28: 'v',
 29: 'w',
 30: 'x',
 31: 'y',
 32: 'z',
 33: 'é',
 34: '’',
 35: '♀',
 36: '♂'}

In [15]:
# Convert each name to a list of integers using char_indices
names_ind = []
for name in names:
    name_ind = []
    for char in name:
        name_ind.append(char_indices[char])
    names_ind.append(name_ind)
        

In [16]:
# First 10 Pokemon names as a list of integers
names_ind[:10]

[[8, 27, 18, 8, 7, 25, 7, 27, 24, 1],
 [15, 28, 31, 25, 7, 27, 24, 1],
 [28, 11, 20, 27, 25, 7, 27, 24, 1],
 [9, 14, 7, 24, 19, 7, 20, 10, 11, 24, 1],
 [9, 14, 7, 24, 19, 11, 18, 11, 21, 20, 1],
 [9, 14, 7, 24, 15, 32, 7, 24, 10, 1],
 [25, 23, 27, 15, 24, 26, 18, 11, 1],
 [29, 7, 24, 26, 21, 24, 26, 18, 11, 1],
 [8, 18, 7, 25, 26, 21, 15, 25, 11, 1],
 [9, 7, 26, 11, 24, 22, 15, 11, 1]]

In [23]:
# Create sequences
sequences = []
for name_ind in names_ind:
    for i in range(1, len(name_ind)):
        sequence = name_ind[:i+1]
        sequences.append(sequence)

In [24]:
# List of sequences
sequences[:20]

[[8, 27],
 [8, 27, 18],
 [8, 27, 18, 8],
 [8, 27, 18, 8, 7],
 [8, 27, 18, 8, 7, 25],
 [8, 27, 18, 8, 7, 25, 7],
 [8, 27, 18, 8, 7, 25, 7, 27],
 [8, 27, 18, 8, 7, 25, 7, 27, 24],
 [8, 27, 18, 8, 7, 25, 7, 27, 24, 1],
 [15, 28],
 [15, 28, 31],
 [15, 28, 31, 25],
 [15, 28, 31, 25, 7],
 [15, 28, 31, 25, 7, 27],
 [15, 28, 31, 25, 7, 27, 24],
 [15, 28, 31, 25, 7, 27, 24, 1],
 [28, 11],
 [28, 11, 20],
 [28, 11, 20, 27],
 [28, 11, 20, 27, 25]]

In [25]:
# Set the max length to be the length of the longest pokemon name (including \n)
maxlen = len(max(names_ind, key=len))
print('Max length:', maxlen)

Max length: 13


In [26]:
# Pad every sequence to be of the same length
from keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences, maxlen=maxlen, padding='pre')

In [27]:
# These sequences include the input and output and are required to have the same length
sequences[:20]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8, 27],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8, 27, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  8, 27, 18,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  8, 27, 18,  8,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  8, 27, 18,  8,  7, 25],
       [ 0,  0,  0,  0,  0,  0,  8, 27, 18,  8,  7, 25,  7],
       [ 0,  0,  0,  0,  0,  8, 27, 18,  8,  7, 25,  7, 27],
       [ 0,  0,  0,  0,  8, 27, 18,  8,  7, 25,  7, 27, 24],
       [ 0,  0,  0,  8, 27, 18,  8,  7, 25,  7, 27, 24,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15, 28],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15, 28, 31],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 15, 28, 31, 25],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 15, 28, 31, 25,  7],
       [ 0,  0,  0,  0,  0,  0,  0, 15, 28, 31, 25,  7, 27],
       [ 0,  0,  0,  0,  0,  0, 15, 28, 31, 25,  7, 27, 24],
       [ 0,  0,  0,  0,  0, 15, 28, 31, 25,  7, 27, 24,  1],
       [ 0,  0,  0,  0, 

In [35]:
# Number of unique characters + 1
vocab_size = len(chars) + 1 # including 0th index
print('Vocab size:', vocab_size)

Vocab size: 37


In [37]:
# Split sequences into inputs and outputs
from keras.utils import to_categorical

X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size) # convert y to one-hot vector

print('X shape:', X.shape)
print('y shape:', y.shape)

X shape: (6032, 12)
y shape: (6032, 37)


In [38]:
# Each output is represented as one-hot vectors of length vocab_size
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [39]:
# Build a RNN
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(vocab_size, 10, input_length=X.shape[1]),
    LSTM(50, dropout=0.2),
    Dense(vocab_size, activation='softmax')]
)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 12, 10)            370       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 37)                1887      
Total params: 14,457
Trainable params: 14,457
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
# Compile and fit the model to X and y
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y, epochs=500)

"""
Note:
Dropout is used so generated names vary from original names (less overfitting). This will 
result in a lower training accuracy which isn't a problem in our case of generating names.
Also, an embedding layer isn't required since characters are more or less independent from
one another.
"""

Epoch 1/500
6032/6032 [==============================] - 8s 1ms/step - loss: 3.0707 - acc: 0.1330
Epoch 2/500
6032/6032 [==============================] - 6s 964us/step - loss: 2.8367 - acc: 0.1790
Epoch 3/500
6032/6032 [==============================] - 6s 964us/step - loss: 2.7623 - acc: 0.2006
Epoch 4/500
6032/6032 [==============================] - 6s 972us/step - loss: 2.7199 - acc: 0.2120
Epoch 5/500
6032/6032 [==============================] - 6s 971us/step - loss: 2.6394 - acc: 0.2293
Epoch 6/500
6032/6032 [==============================] - 6s 963us/step - loss: 2.5811 - acc: 0.2273
Epoch 7/500
6032/6032 [==============================] - 6s 965us/step - loss: 2.5608 - acc: 0.2263
Epoch 8/500
6032/6032 [==============================] - 6s 965us/step - loss: 2.5497 - acc: 0.2293
Epoch 9/500
6032/6032 [==============================] - 6s 967us/step - loss: 2.5394 - acc: 0.2266
Epoch 10/500
6032/6032 [==============================] - 6s 962us/step - loss: 2.5242 - acc: 0.2280
E

6032/6032 [==============================] - 6s 947us/step - loss: 2.1666 - acc: 0.3128
Epoch 83/500
6032/6032 [==============================] - 6s 941us/step - loss: 2.1539 - acc: 0.3200
Epoch 84/500
6032/6032 [==============================] - 6s 959us/step - loss: 2.1515 - acc: 0.3193
Epoch 85/500
6032/6032 [==============================] - 6s 955us/step - loss: 2.1485 - acc: 0.3203
Epoch 86/500
6032/6032 [==============================] - 6s 957us/step - loss: 2.1437 - acc: 0.3163
Epoch 87/500
6032/6032 [==============================] - 6s 975us/step - loss: 2.1381 - acc: 0.3218
Epoch 88/500
6032/6032 [==============================] - 6s 975us/step - loss: 2.1368 - acc: 0.3148
Epoch 89/500
6032/6032 [==============================] - 6s 978us/step - loss: 2.1330 - acc: 0.3234
Epoch 90/500
6032/6032 [==============================] - 6s 963us/step - loss: 2.1276 - acc: 0.3219
Epoch 91/500
6032/6032 [==============================] - 6s 946us/step - loss: 2.1299 - acc: 0.3200
Epo

6032/6032 [==============================] - 6s 986us/step - loss: 1.8976 - acc: 0.3929
Epoch 163/500
6032/6032 [==============================] - 6s 985us/step - loss: 1.8902 - acc: 0.3936
Epoch 164/500
6032/6032 [==============================] - 6s 985us/step - loss: 1.9021 - acc: 0.3951
Epoch 165/500
6032/6032 [==============================] - 6s 975us/step - loss: 1.8776 - acc: 0.3937
Epoch 166/500
6032/6032 [==============================] - 6s 972us/step - loss: 1.8828 - acc: 0.3994
Epoch 167/500
6032/6032 [==============================] - 6s 973us/step - loss: 1.8902 - acc: 0.3912
Epoch 168/500
6032/6032 [==============================] - 6s 970us/step - loss: 1.8812 - acc: 0.3985
Epoch 169/500
6032/6032 [==============================] - 6s 987us/step - loss: 1.8860 - acc: 0.4005
Epoch 170/500
6032/6032 [==============================] - 6s 980us/step - loss: 1.8734 - acc: 0.3989
Epoch 171/500
6032/6032 [==============================] - 6s 988us/step - loss: 1.8762 - acc: 0

6032/6032 [==============================] - 6s 990us/step - loss: 1.7279 - acc: 0.4465
Epoch 243/500
6032/6032 [==============================] - 6s 988us/step - loss: 1.7272 - acc: 0.4451
Epoch 244/500
6032/6032 [==============================] - 6s 986us/step - loss: 1.7333 - acc: 0.4403
Epoch 245/500
6032/6032 [==============================] - 6s 993us/step - loss: 1.7394 - acc: 0.4390
Epoch 246/500
6032/6032 [==============================] - 6s 986us/step - loss: 1.7323 - acc: 0.4458
Epoch 247/500
6032/6032 [==============================] - 6s 994us/step - loss: 1.7222 - acc: 0.4446
Epoch 248/500
6032/6032 [==============================] - 6s 999us/step - loss: 1.7275 - acc: 0.4406
Epoch 249/500
6032/6032 [==============================] - 7s 1ms/step - loss: 1.7273 - acc: 0.4395
Epoch 250/500
6032/6032 [==============================] - 7s 1ms/step - loss: 1.7230 - acc: 0.4461
Epoch 251/500
6032/6032 [==============================] - 6s 1ms/step - loss: 1.7250 - acc: 0.4428


6032/6032 [==============================] - 6s 977us/step - loss: 1.6281 - acc: 0.4718
Epoch 324/500
6032/6032 [==============================] - 6s 970us/step - loss: 1.6422 - acc: 0.4738
Epoch 325/500
6032/6032 [==============================] - 6s 1ms/step - loss: 1.6300 - acc: 0.4745
Epoch 326/500
6032/6032 [==============================] - 7s 1ms/step - loss: 1.6325 - acc: 0.4712
Epoch 327/500
6032/6032 [==============================] - 6s 1ms/step - loss: 1.6157 - acc: 0.4778
Epoch 328/500
6032/6032 [==============================] - 6s 1ms/step - loss: 1.6317 - acc: 0.4683
Epoch 329/500
6032/6032 [==============================] - 6s 1ms/step - loss: 1.6277 - acc: 0.4703A: 0s - loss: 1.6305 - acc: 0.469
Epoch 330/500
6032/6032 [==============================] - 6s 1ms/step - loss: 1.6262 - acc: 0.4703
Epoch 331/500
6032/6032 [==============================] - 6s 985us/step - loss: 1.6289 - acc: 0.4733
Epoch 332/500
6032/6032 [==============================] - 6s 977us/step - 

6032/6032 [==============================] - 6s 964us/step - loss: 1.5649 - acc: 0.4910
Epoch 404/500
6032/6032 [==============================] - 6s 968us/step - loss: 1.5641 - acc: 0.5002
Epoch 405/500
6032/6032 [==============================] - 6s 971us/step - loss: 1.5619 - acc: 0.4992
Epoch 406/500
6032/6032 [==============================] - 6s 973us/step - loss: 1.5578 - acc: 0.4934
Epoch 407/500
6032/6032 [==============================] - 6s 974us/step - loss: 1.5633 - acc: 0.4834
Epoch 408/500
6032/6032 [==============================] - 6s 967us/step - loss: 1.5589 - acc: 0.4955
Epoch 409/500
6032/6032 [==============================] - 6s 988us/step - loss: 1.5681 - acc: 0.4952
Epoch 410/500
6032/6032 [==============================] - 6s 971us/step - loss: 1.5501 - acc: 0.4987
Epoch 411/500
6032/6032 [==============================] - 6s 968us/step - loss: 1.5634 - acc: 0.4959
Epoch 412/500
6032/6032 [==============================] - 6s 972us/step - loss: 1.5663 - acc: 0

6032/6032 [==============================] - 6s 984us/step - loss: 1.5213 - acc: 0.5066
Epoch 484/500
6032/6032 [==============================] - 6s 975us/step - loss: 1.5149 - acc: 0.5060
Epoch 485/500
6032/6032 [==============================] - 6s 979us/step - loss: 1.5190 - acc: 0.5015
Epoch 486/500
6032/6032 [==============================] - 6s 1ms/step - loss: 1.5003 - acc: 0.5106
Epoch 487/500
6032/6032 [==============================] - 6s 1ms/step - loss: 1.5258 - acc: 0.5007
Epoch 488/500
6032/6032 [==============================] - 6s 988us/step - loss: 1.5098 - acc: 0.5046
Epoch 489/500
6032/6032 [==============================] - 6s 1ms/step - loss: 1.5218 - acc: 0.4922
Epoch 490/500
6032/6032 [==============================] - 6s 1ms/step - loss: 1.5132 - acc: 0.5017
Epoch 491/500
6032/6032 [==============================] - 6s 996us/step - loss: 1.5080 - acc: 0.5106
Epoch 492/500
6032/6032 [==============================] - 6s 999us/step - loss: 1.5074 - acc: 0.5045
Ep

In [44]:
import sys

# Prompt user input
text = input('Insert text: ')

# Convert inputted text to a list of character indices
X_user = [char_indices[char] for char in text]


print('======== Generated name ========')
sys.stdout.write(text)

# Pad X_user to have the same length as our training examples
X_user = pad_sequences([X_user], maxlen=X.shape[1], padding='pre')

# Repeatedly generate a new character and update X_user to include it
for i in range(maxlen):
    # Prediction on X_user
    y_pred = model.predict(X_user)

    # Sample from y_pred and convert to character
    sample = np.random.choice(vocab_size, 1, p = y_pred.ravel())
    next_char = indices_char[sample[0]]
    
    # Break out of loop if the predicted character is a newline
    if next_char == '\n':
        break
    
    # Print the sampled character and update X_user to include it
    sys.stdout.write(next_char)
    X_user = (np.append(X_user[0][1:], (sample[0]))).reshape(1,-1)


Insert text: ch
======== Generated name ========
charicade